## Knihovny a moduly pro matematické výpočty

### 0. Příprava

#### 0.1. Načtení knihoven 

In [2]:
# Knihovny obsahující testovaný obsah
import numpy as np
import statistics
import math
#-----------------
# Pomocné knihovny
import timeit
from random import randint
from typing import Callable, Iterable


#### 0.2. Vytvoření testovací funkce

In [9]:
def function_testing(test_func:Callable, test_data:any) -> float:
    testing_code = lambda: test_func(test_data)

    times = timeit.repeat(testing_code, number=1000)

    return sum(times) / len(times)

### 1. Kumulativní suma

#### 1.1. Standardní knihovna

In [2]:
def cumsum_std(unsummed:Iterable) -> Iterable | list:
    if not isinstance(unsummed, list):
            unsummed = list(unsummed)

    for index in range(1, len(unsummed)):
        unsummed[index] += unsummed[index - 1]
        
    return unsummed

In [3]:
cumsum_std((4,5,3,5,4))

[4, 9, 12, 17, 21]

#### 1.2. Specializovaná knihovna

In [3]:
def cumsum_spec(unsummed:Iterable) -> "np.array":
    if not isinstance(unsummed, np.ndarray):
        unsummed = np.array(unsummed)
    return unsummed.cumsum()

In [4]:
cumsum_spec((4,5,3,5,4))

array([ 4,  9, 12, 17, 21])

#### 1.3. Testování
Poznámka: Každá funkce má lepší výkon pro specifický typ vstupu, takže asi nejlepší bude testovat rychlost pro více typů aby šlo výkon správně porovnat.

In [93]:
example_data_cumsum_list = [randint(1,10) for x in range(10000)]
example_data_cumsum_array = np.array([randint(1,10) for x in range(10000)])
example_data_cumsum_tuple = tuple([randint(1,10) for x in range(10000)])

In [100]:
print(function_testing(cumsum_std, example_data_cumsum_list))
print(function_testing(cumsum_spec, example_data_cumsum_list))

3.8987202400050593
10.317593379999744


In [101]:
print(function_testing(cumsum_std, example_data_cumsum_array))
print(function_testing(cumsum_spec, example_data_cumsum_array))

C:\Users\Matej\AppData\Local\Temp\ipykernel_20444\2692247240.py:8: RuntimeWarning: overflow encountered in scalar add
  unsummed[index] += unsummed[index - 1]


5.1411001200089235
0.01826794000226073


In [106]:
print(function_testing(cumsum_std, example_data_cumsum_tuple))
print(function_testing(cumsum_spec, example_data_cumsum_tuple))

1.0186178200005087
0.4902250599930994


### 2. Determinant

#### 2.1. Standardní knihovna

In [59]:
def downslice(matrix:list, index:int) -> list:
    if len(matrix) == index:
        selection = lambda row: []
    else:
        selection = lambda row: row[index+1:]
    
    unrow = matrix[1:]
    uncolumn = [row[:index] + selection(row=row) for row in unrow]

    return uncolumn

def square_check(matrix:list):
    width = len(matrix)
    for row in matrix:
        if width != len(row):
            return False
        
    return True


def determinant_std(matrix:list) -> int | float:
    assert square_check(matrix), "Matrix must be of a square shape."

    if len(matrix) == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    
    first_row = matrix[0]

    one_expansion = lambda index, num: (-1)**index * num * determinant_std(downslice(matrix, index))

    expansion_values = [one_expansion(index=index, num=num) for index, num in enumerate(first_row)]

    return sum(expansion_values)


In [65]:
myList = [
    [1,2,3,6],
    [4,5,6,16],
    [7,5,9,21],
    [10,11,12,33]
]

determinant_std(myList)

54

#### 2.2. Specializovaná knihovna

In [60]:
determinant_spec = lambda matrix: np.linalg.det(matrix)

In [66]:
determinant_spec(myList)

54.00000000000001

#### 2.3. Testování

In [69]:
example_determinant = [ [randint(-15,15) for x in range(6)] for y in range(6)]
example_determinant

[[-10, 12, -10, -10, 4, 14],
 [-9, -8, 0, 14, 12, -12],
 [-2, 1, -12, -2, -8, -7],
 [12, 1, -12, -11, 14, 0],
 [-9, 2, -6, 1, 4, -13],
 [1, 3, 3, 4, -12, 6]]

In [70]:
print(function_testing(determinant_std, example_determinant))
print(function_testing(determinant_spec, example_determinant))

1.0220108999870718
0.009488440002314746


### 3. Transponovaná matice

#### 3.1. Standardní knihovna

In [71]:
def transposed_std(matrix:list) -> list:
    return [ [matrix[y][x] for y in range(len(matrix)) ] for x in range(len(matrix[0]))]

In [72]:
transposed_std(myList)

[[1, 4, 7, 10], [2, 5, 5, 11], [3, 6, 9, 12], [6, 16, 21, 33]]

#### 3.2. Specializovaná knihovna

In [73]:
transposed_spec = lambda matrix: np.matrix.transpose(matrix)

In [75]:
myArray = np.array(myList)
transposed_spec(myArray)

array([[ 1,  4,  7, 10],
       [ 2,  5,  5, 11],
       [ 3,  6,  9, 12],
       [ 6, 16, 21, 33]])

#### 3.3. Testování

In [78]:
example_transposition_list = [ [randint(-15,15) for x in range(25)] for y in range(25)]
example_transposition_array = np.array(example_transposition_list)
example_transposition_array

array([[ 10,  -6, -14, -11,  -4,   2,   2,  11,   1,   7,  -8,  11,  -4,
         -2,  -3,  14,  -4,   1,   0,  10,  10,   1, -13,  14, -15],
       [ -7,  -3, -14,   2,  -8, -14,  -9,  12, -14, -11,  -1, -12, -14,
         10,  -3,  15,   1,   7,   6,   0, -13,   0,   8,   3, -10],
       [ -7,  -7,  10,   9,   7,  -9,  -6,  -6,  -1,   0,  -3,   3,   5,
         -8,  13,  -6,  14,  13, -15,   4, -14,  13, -11,  -4,   9],
       [-15,   5,  11,  15,  12,  14,   9,  -3,   5,  -8, -13,  15,   5,
          9,  -9, -13,   9,  -3, -10,   2,  13,   9,  -8,   9, -14],
       [  4,   3,  -7,  -4,  -9,  -5,  -3, -12, -14,   2,  -4,   2,  -9,
         -4, -11,   0,  -8,  10,   6,   4,  -4,   1,  14,  -1, -14],
       [ 15, -15,  15,   9, -14,   5,  12,   6,  11,  14,   4,  14,   3,
         12,  14, -10, -10,   3,  -8,  12,   0,  15,  -5,   0,  14],
       [ -4,  15,   0,  -9,   7,   4, -14, -11,  10,  10,  14,   2,  10,
         13, -10,   5,  13, -13, -13,   5,   1,   7,   8, -11,   5],
      

In [79]:
print(function_testing(transposed_std, example_transposition_list))
print(function_testing(transposed_spec, example_transposition_array))

0.03428053999668919
0.00021081999875605106


### 4. Směrodatná odchylka

#### 4.1. Standardní knihovna

In [34]:
def std_dev_std(data:Iterable) -> float:
    average_val = sum(data) / len(data)
    variance = sum([(data[index] - average_val)**2 for index in range(len(data))]) * (1 / len(data))
    return variance**(1/2)

#### 4.2. Specializovaná knihovna

In [31]:
def std_dev_spec(data:Iterable) -> float:
    return statistics.pstdev(data)

#### 4.3. Testování

In [37]:
example_std_dev = [randint(1, 50) for x in range(20)]

In [38]:
print(function_testing(std_dev_std, example_std_dev))
print(function_testing(std_dev_spec, example_std_dev))


0.006168959999922663
0.026250360009726137


### 5. Kombinační číslo

#### 5.1. Standardní knihovna

In [2]:
def comb_std(parameter_tuple:tuple) -> int:
    element_count = parameter_tuple[0]
    pick_count = parameter_tuple[1]
    variation_elems = range(element_count, element_count - pick_count, -1)
    repeat_cases_elems = range(1, pick_count + 1)

    variation = 1
    repeat_cases = 1
    for index in range(pick_count):
        variation *= variation_elems[index]
        repeat_cases *= repeat_cases_elems[index]

    return int(variation / repeat_cases)

In [3]:
comb_std((10, 4))

210

#### 5.2. Specializovaná knihovna

In [4]:
def comb_spec(parameter_tuple:tuple) -> int:
    element_count = parameter_tuple[0]
    pick_count = parameter_tuple[1]
    return math.comb(element_count, pick_count)

In [7]:
comb_spec((10, 4))

210

#### 5.3. Testování

In [13]:
print(function_testing(comb_std, (1000, 45)))
print(function_testing(comb_spec, (1000, 45)))

0.015082280000206083
0.001779559999704361
